In [1]:
# default_exp spec

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
#export
import re
from typing import List, Tuple

from numpy import ndarray
from pandas import DataFrame, Series
from sentence_transformers import SentenceTransformer, models
from spacy_stanza import StanzaLanguage

In [4]:
#export
def gsheet_to_df(worksheet) -> DataFrame:
    df = DataFrame(worksheet.get_all_values())
    df.columns = df.iloc[0]
    df = df.iloc[1:]
    return df

In [14]:
#export
GSHEET = "GSHEET"
TRANSFORMER = "TRANSFORMER"
SENTENCE_TRANSFORMER = "SENTENCE_TRANSFORMER"
PUBMED_IDS = "PUBMED_IDS"
PUBMED_CONTENT = "PUBMED_CONTENT"
EMAIL = "EMAIL"
SPACY_MODEL = "SPACY_MODEL"
STANZA_MODEL = "STANZA_MODEL"

# for t in TYPES:  # PYTHON MAGIC
#     exec("%s=str('%s')" % (t, t))

"""
? Dynamically create Types in NameSpace aka TRANSFORMER = "TRANSFORMER"
"""

'\n? Dynamically create Types in NameSpace aka TRANSFORMER = "TRANSFORMER"\n'

In [6]:
#export
def _transformer_to_sentence_transformer(transformer_model):
    pooling_model = models.Pooling(
        transformer_model.get_word_embedding_dimension(),
        pooling_mode_mean_tokens=True,
        pooling_mode_cls_token=True,
        pooling_mode_max_tokens=False,
    )

    return SentenceTransformer(modules=[transformer_model, pooling_model])

In [7]:
# (Source, Target) -> Transform Function
#export
TRANSFORMS = {
    (STANZA_MODEL, SPACY_MODEL): StanzaLanguage,
    (str, list): lambda string: [string],
    (Tuple[str], List[str]): list,
    (GSHEET, DataFrame): gsheet_to_df,
    (Series, list): Series.to_list,
    (ndarray, list): list,
    (TRANSFORMER, SENTENCE_TRANSFORMER): _transformer_to_sentence_transformer,
}

In [8]:
#export
SPEC = {
    GSHEET: re.compile(".*docs.google.com\/spreadsheets.*"),
    SPACY_MODEL: re.compile(
        "(zh|da|nl|en|fr|de|el|it|ja|lt|nb|pl|pt|ro|es|xx)[_(core|ent|ner)_(web|news|wiki|sci|craft|jnlpba|bc5cdr|bionlp13cg)_(sm|md|lg)]*$"
    ),
    STANZA_MODEL: re.compile("stanza"),
    PUBMED_IDS: re.compile(".*pubmed.ncbi.nlm.nih.gov.*"),
    TRANSFORMER: re.compile("TRANSFORMER:.*"),
}

In [9]:
#export
def infer_type(shape):
    """ What types match this shape? """
    match = [key for key, value in SPEC.items() if SPEC[key].match(shape)]

    if len(match) > 1:
        raise Exception(
            "Found multiple inferences for the shape you put in. Please put the input_type =EMAIL or something as a keyword argument"
        )

    return match[0]